In [1]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

2023-12-09 16:58:05.817036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 16:58:06.820081: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-09 16:58:06.820171: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-09 16:58:06.820184: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
class TextDataset(Dataset):
    def __init__(self, text_file, tokenizer):
        self.tokenizer = tokenizer
        self.data = []
        with open(text_file, 'r') as file:
            for line in file:
                self.data.append(tokenizer(line, truncation=True, padding='max_length', max_length=128))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
dataset = TextDataset('fr_part_1.txt', tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

KeyboardInterrupt: 

In [3]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss / len(dataloader)}")

AttributeError: 'list' object has no attribute 'to'

In [ ]:
# Supposons que "sentence" est la phrase que vous voulez tester, avec un token masqué
sentence = "J'aime le <mask>."

# Préparation de l'entrée
input = tokenizer.encode(sentence, return_tensors="pt").to(device)

# Prédiction du token masqué
output = model(input)
prediction = torch.argmax(output.logits, dim=2)

# Conversion du token prédit en mot
predicted_token = tokenizer.convert_ids_to_tokens(prediction[0].tolist())
print(predicted_token)

---
DELIMITATION
---

In [1]:
from transformers import RobertaTokenizerFast, RobertaForMaskedLM,AutoTokenizer 
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from utilis.utilis import PreProcessing
import torch
from torch.utils.data import Dataset

2023-12-10 13:19:11.021541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 13:19:12.033517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-10 13:19:12.033612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-10 13:19:12.033624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
file_path = 'fr_part_1_short.txt'
pre_process = PreProcessing(file_path)
input_texts = pre_process.read_dataset() 
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_len=512)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [3]:
# class MyDataset(Dataset):
#     def __init__(self, texts, tokenizer):
#         self.tokenizer = tokenizer
#         self.inputs = []
#         self.targets = []

#         for text in texts:
#             inputs = tokenizer(text, truncation=True, max_length=512, padding='max_length') 
#             self.inputs.append(inputs.input_ids)
#             self.targets.append(inputs.input_ids)

#     def __len__(self):
#         return len(self.inputs)

#     def __getitem__(self, idx):
#         item = {"input_ids": self.inputs[idx], "labels": self.targets[idx]}
#         return item
class MyDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size:int = 128):
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        for text in texts:
            inputs = tokenizer(text, truncation=True, max_length=512, padding='max_length') 
            examples = {"input_ids": inputs.input_ids}
            grouped_examples = self.group_texts(examples, block_size)
            self.inputs.extend(grouped_examples["input_ids"])
            self.targets.extend(grouped_examples["labels"])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item = {"input_ids": self.inputs[idx], "labels": self.targets[idx]}
        return item

    def group_texts(self,examples, block_size):
        # Concatenate all texts.
        concatenated_examples = {k: examples[k] for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
        if total_length >= block_size:
            total_length = (total_length // block_size) * block_size
        # Split by chunks of block_size.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

dataset = MyDataset(input_texts, tokenizer)

In [4]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./roberta-mlm",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=64,
    save_steps=10,
    save_total_limit=50,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [5]:
trainer_result = trainer.train()
trainer.save_model("./roberta-retrained")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [10]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./roberta-retrained",
    tokenizer="roberta-base"
)
fill_mask("Je prépare un gateau au <mask>.")

[{'score': 0.20697994530200958,
  'token': 4532,
  'token_str': ' maximum',
  'sequence': 'Je prépare un gateau au maximum.'},
 {'score': 0.06615154445171356,
  'token': 5059,
  'token_str': ' blog',
  'sequence': 'Je prépare un gateau au blog.'},
 {'score': 0.05863066762685776,
  'token': 1082,
  'token_str': ' site',
  'sequence': 'Je prépare un gateau au site.'},
 {'score': 0.03209709748625755,
  'token': 914,
  'token_str': ' match',
  'sequence': 'Je prépare un gateau au match.'},
 {'score': 0.027383187785744667,
  'token': 11739,
  'token_str': ' blend',
  'sequence': 'Je prépare un gateau au blend.'}]

In [24]:
import matplotlib.pyplot as plt

train_metrics = trainer_result.metrics

print(train_metrics['train_loss'])

plt.figure(figsize=(12, 6))
plt.plot(train_metrics['accuracy'])
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.show()

# Tracer la loss
plt.figure(figsize=(12, 6))
plt.plot(train_metrics['loss'])
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.tight_layout()
plt.show()


1.2287200927734374


KeyError: 'accuracy'

<Figure size 1200x600 with 0 Axes>

In [25]:
model_path = "CamemBERT/CamemBERT/roberta-mlm" 
model = RobertaForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

sentence = "Le chat <mask> sur le mur."

# Tokenisation de la phrase avec le tokenizer
inputs = tokenizer(sentence, return_tensors="pt")

# Obtenir les positions du masque dans la phrase
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)

# Faire une prédiction pour le mot masqué
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits[mask_token_index]

# Obtenir les 5 prédictions les plus probables
top_k = 5
top_predictions = torch.topk(predictions[0, mask_token_index], k=top_k, dim=1)

# Afficher les prédictions
for i in range(top_k):
    predicted_index = top_predictions.indices[0][i].item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(f"Prédiction {i+1}: '{predicted_token}' avec une probabilité de {top_predictions.values[0][i].item():.4f}")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'CamemBERT/CamemBERT/roberta-mlm'. Use `repo_type` argument if needed.

In [8]:
import torch
test_sentence = "Je prépzre un gateau au [MASK]."
input = tokenizer.encode(test_sentence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(tokenizer.decode([token]))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)